In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import soundfile as sf

##Return audio features
def feature_extraction(file_name):
    X, sample_rate = librosa.load(file_name)
    if X.ndim > 1:
        X = X[:,0]
    X = X.T

    # Get features
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0) #40 values
    zcr = np.mean(librosa.feature.zero_crossing_rate)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0)
    return mfccs, chroma,mel,contrast, tonnetz,zcr

In [150]:
def parse_audio_files(parent_dir,file_ext):
    features, labels = np.empty((0,193)), np.empty(0)
    for label, sub_dir in enumerate(parent_dir):
        for file_name in glob.glob(os.path.join(audio_directories, file_ext)):
            try:
                mfccs, chroma, mel, contrast, tonnetz = feature_extraction(file_name)
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue

            extracted_features = np.hstack([mfccs,chroma, mel, contrast, tonnetz])
            features = np.vstack([features, extracted_features])
            labels = np.append(labels, label)
        print("Extracted features from %s, done" % (sub_dir))

    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode
parent_dir='/content/drive/MyDrive/audio'
audio_directories = [os.path.join(parent_dir, subdir) for subdir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, subdir))]
audio_directories.sort()
print('Audio Classes: ', audio_directories)

Audio Classes:  ['/content/drive/MyDrive/audio/Airplane', '/content/drive/MyDrive/audio/Footsteps', '/content/drive/MyDrive/audio/chinsaw', '/content/drive/MyDrive/audio/chirping_birds', '/content/drive/MyDrive/audio/door_wood_knock', '/content/drive/MyDrive/audio/fire_works', '/content/drive/MyDrive/audio/hen', '/content/drive/MyDrive/audio/pouring_water', '/content/drive/MyDrive/audio/sheep', '/content/drive/MyDrive/audio/thunderstrom']


In [ ]:
features,labels=parse_audio_files(parent_dir,'*.wav')
print(features)
np.save('feat.npy', features)
np.save('label.npy', labels)


In [139]:
labels = np.load('label.npy')
print(labels)
label_classes = np.array(['Airplane', 'Footsteps', 'chinsaw', 'chirping_birds', 'door_wood_knock', 'fire_works', 'hen', 'pouring_water', 'sheep', 'thunderstrom'])
print(label_classes)

[]
['Airplane' 'Footsteps' 'chinsaw' 'chirping_birds' 'door_wood_knock'
 'fire_works' 'hen' 'pouring_water' 'sheep' 'thunderstrom']


In [153]:
features= np.load('feat.npy')
print(len(features))
print(features)

0
[]


In [40]:
import pandas as pd
df = pd.DataFrame(features)
df['Audio class'] = pd.Categorical.from_codes(labels, label_classes)

df[[0,1,2,3,4,5,6,7,8,9,10,'Audio class']]

,0,1,2,3,4,5,6,7,8,9,10,Audio class


In [ ]:
import numpy as np
import sklearn
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
X = np.load('feat.npy')
y = np.load('label.npy').ravel()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))
svm_clf = SVC(C=28.0, gamma = 0.00001, decision_function_shape="ovr")

svm_clf.fit(X_train, y_train)
y_predict = svm_clf.predict(X_test)
acc = svm_clf.score(X_test, y_test)
print
print("accuracy=%0.3f" %acc)